In [1]:
import os
import re
import math
import cv2
import pandas as pd
import numpy as np

from tqdm import tqdm
from glob import glob
from tools.settings import *
from keras.models import Sequential
from keras.preprocessing import image
from keras.layers import Dense, Dropout
from keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import to_categorical
from tools.train_val_test_spliter import split

In [2]:
# To split dataset if already splits folder already exits no need to run it
split()

Splitting the ginen dataset into Train Test=0.3 Validation=0.2
Done


In [3]:
train = pd. read_csv(os.path.join(dataset_path, "train.csv"))
test = pd. read_csv(os.path.join(dataset_path, "test.csv"))
val = pd. read_csv(os.path.join(dataset_path, "val.csv"))

In [4]:
train.head()

,Video_url,action
0,H:/GitHub/real-time-action-recognition-from-vi...,punch
1,H:/GitHub/real-time-action-recognition-from-vi...,kick
2,H:/GitHub/real-time-action-recognition-from-vi...,punch
3,H:/GitHub/real-time-action-recognition-from-vi...,punch
4,H:/GitHub/real-time-action-recognition-from-vi...,slap


In [5]:
test.head()

,Video_url,action
0,H:/GitHub/real-time-action-recognition-from-vi...,punch
1,H:/GitHub/real-time-action-recognition-from-vi...,punch
2,H:/GitHub/real-time-action-recognition-from-vi...,punch
3,H:/GitHub/real-time-action-recognition-from-vi...,kick
4,H:/GitHub/real-time-action-recognition-from-vi...,punch


In [6]:
val.head()

,Video_url,action
0,H:/GitHub/real-time-action-recognition-from-vi...,punch
1,H:/GitHub/real-time-action-recognition-from-vi...,punch
2,H:/GitHub/real-time-action-recognition-from-vi...,slap
3,H:/GitHub/real-time-action-recognition-from-vi...,punch
4,H:/GitHub/real-time-action-recognition-from-vi...,slap


In [7]:
print(train.shape)
print(test.shape)
print(val.shape)

(78, 2)
(44, 2)
(29, 2)


In [9]:
def convert_to_frame(data, folder_name):
    '''
    Generated filenames format dataset_path/folder_name/video_name_frame{number}_action.jpg
    '''
    directory = os.path.join(dataset_path, folder_name)
    if not os.path.exists(directory):
        os.makedirs(directory)
    for i in tqdm(range(data.shape[0])):
        video_file = data['Video_url'][i]
        action = data['action'][i]
        video_name_list = video_file.split('/')[-1].split('.')
        video_name_list = video_name_list[:-1]
        video_name = ""
        for n in video_name_list:
            video_name += n
        # capturing the video from the given path
        capture = cv2.VideoCapture(video_file) 
        #frame rate
        frame_rate = capture.get(5)
        count = 0
        while(capture.isOpened()):
            #current frame number
            frame_id = capture.get(1) 
            read_correctly, frame = capture.read()
            if not read_correctly:
                break
            if (frame_id % math.floor(frame_rate) == 0):
                # storing the frames in a new folder named train_1
                filename = directory + "/" + video_name + "_frame{}_".format(count) + action +".jpg"
                count += 1
                cv2.imwrite(filename, frame)
        capture.release()
    print("Successfully Converted")

In [10]:
convert_to_frame(train, train_frames_path_name)

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [01:04<00:00,  1.20it/s]

Successfully Converted


In [11]:
convert_to_frame(val, val_frames_path_name)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:23<00:00,  1.26it/s]

Successfully Converted


In [12]:
def create_paths_csv(directory, file_name):
    images = os.listdir(directory)
    images_path_list = []
    images_action_list = [] 
    for image in images:
        images_path_list.append(directory + image)
        images_action_list.append(image.split('.')[0].split('_')[-1])
    df = pd.DataFrame()
    df['image'] = images_path_list
    df['action'] = images_action_list
    print(os.path.join(dataset_path, file_name+'.csv'))
    df.to_csv(os.path.join(dataset_path, file_name+'.csv'), index=False)

In [13]:
create_paths_csv(train_frames_path, train_frames_path_name)

H:/GitHub/real-time-action-recognition-from-video-footage/dataset\train_frames.csv


In [14]:
create_paths_csv(val_frames_path, val_frames_path_name)

H:/GitHub/real-time-action-recognition-from-video-footage/dataset\val_frames.csv


In [15]:
train_image = pd.read_csv(os.path.join(dataset_path, 'train_frames.csv'))
train_image.head()

,image,action
0,H:/GitHub/real-time-action-recognition-from-vi...,kick
1,H:/GitHub/real-time-action-recognition-from-vi...,kick
2,H:/GitHub/real-time-action-recognition-from-vi...,kick
3,H:/GitHub/real-time-action-recognition-from-vi...,kick
4,H:/GitHub/real-time-action-recognition-from-vi...,kick


In [16]:
print(train_image.shape)

(423, 2)


In [17]:
val_image = pd.read_csv(os.path.join(dataset_path, 'val_frames.csv'))
val_image.head()

,image,action
0,H:/GitHub/real-time-action-recognition-from-vi...,kick
1,H:/GitHub/real-time-action-recognition-from-vi...,kick
2,H:/GitHub/real-time-action-recognition-from-vi...,kick
3,H:/GitHub/real-time-action-recognition-from-vi...,kick
4,H:/GitHub/real-time-action-recognition-from-vi...,kick


In [18]:
print(val_image.shape)

(161, 2)


In [19]:
action_values = list(train_image['action'].unique())
action_values

['kick', 'punch', 'slap']

In [20]:
def create_class_columns(df):
    for value in action_values:
        df[value] = np.where(df['action'].str.contains(value), 1, 0)
    df.drop('action', axis='columns', inplace=True)

In [21]:
create_class_columns(train_image)
train_image.head()

,image,kick,punch,slap
0,H:/GitHub/real-time-action-recognition-from-vi...,1,0,0
1,H:/GitHub/real-time-action-recognition-from-vi...,1,0,0
2,H:/GitHub/real-time-action-recognition-from-vi...,1,0,0
3,H:/GitHub/real-time-action-recognition-from-vi...,1,0,0
4,H:/GitHub/real-time-action-recognition-from-vi...,1,0,0


In [22]:
create_class_columns(val_image)
val_image.head()

,image,kick,punch,slap
0,H:/GitHub/real-time-action-recognition-from-vi...,1,0,0
1,H:/GitHub/real-time-action-recognition-from-vi...,1,0,0
2,H:/GitHub/real-time-action-recognition-from-vi...,1,0,0
3,H:/GitHub/real-time-action-recognition-from-vi...,1,0,0
4,H:/GitHub/real-time-action-recognition-from-vi...,1,0,0


In [23]:
def convert_to_array_and_split(image_data):
    image_value = []
    for i in tqdm(range(image_data.shape[0])):
        img = image.load_img(image_data['image'][i], target_size=(224,224,3))
        img = image.img_to_array(img)
        # normalizing the pixel value
        img = img / 255
        image_value.append(img)

    X = np.array(image_value)
    y = image_data
    y.drop('image', axis='columns', inplace=True)
    return X, y

In [24]:
X_train, y_train = convert_to_array_and_split(train_image)
print(X_train.shape)

100%|████████████████████████████████████████████████████████████████████████████████| 423/423 [00:04<00:00, 84.95it/s]


(423, 224, 224, 3)


In [25]:
X_val, y_val = convert_to_array_and_split(val_image)
print(X_val.shape)

100%|████████████████████████████████████████████████████████████████████████████████| 161/161 [00:01<00:00, 84.92it/s]


(161, 224, 224, 3)


In [26]:
y_train.head()

,kick,punch,slap
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [46]:
y_val.head()

,kick,punch,slap
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [28]:
'''This model was trained on a dataset that has 1,000 classes. 
include_top = False will remove the last layer of this model so that we can tune it as per our need.
'''
base_model = VGG16(weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 28s 0us/step


In [29]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(423, 7, 7, 512)

In [30]:
X_val = base_model.predict(X_val)
X_val.shape

(161, 7, 7, 512)

In [50]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(423, 7*7*512)
X_val = X_val.reshape(161, 7*7*512)

In [51]:
# normalizing the pixel values
max_pixel = X_train.max()
X_train = X_train / max_pixel
X_val = X_val / max_pixel
print(X_train.shape)
print(X_val.shape)

(423, 25088)
(161, 25088)


In [52]:
# The input shape will be 25,088
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)              

In [53]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_weight = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [54]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[mcp_weight], batch_size=128)

Epoch 1/50
4/4 [==============================] - 4s 884ms/step - loss: 1.0788 - accuracy: 0.4326 - val_loss: 0.9324 - val_accuracy: 0.5590
Epoch 2/50
4/4 [==============================] - 1s 196ms/step - loss: 1.2061 - accuracy: 0.4610 - val_loss: 0.9641 - val_accuracy: 0.5590
Epoch 3/50
4/4 [==============================] - 1s 188ms/step - loss: 1.1072 - accuracy: 0.4704 - val_loss: 0.9579 - val_accuracy: 0.5590
Epoch 4/50
4/4 [==============================] - 1s 181ms/step - loss: 1.0766 - accuracy: 0.4870 - val_loss: 0.9592 - val_accuracy: 0.5590
Epoch 5/50
4/4 [==============================] - 4s 909ms/step - loss: 1.0330 - accuracy: 0.4941 - val_loss: 0.9307 - val_accuracy: 0.5466
Epoch 6/50
4/4 [==============================] - 4s 967ms/step - loss: 0.9523 - accuracy: 0.5508 - val_loss: 0.8782 - val_accuracy: 0.5652
Epoch 7/50
4/4 [==============================] - 4s 1s/step - loss: 0.8945 - accuracy: 0.5887 - val_loss: 0.8412 - val_accuracy: 0.5901
Epoch 8/50
4/4 [=======

In [56]:
model.load_weights("weight.hdf5")
# model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
# model.summary()

In [57]:
from scipy import stats as s
predict = []
actual = []
if not os.path.exists(test_frames_path):
    os.makedirs(test_frames_path)

for i in tqdm(range(test.shape[0])):
    video_file = test['Video_url'][i]
    action = test['action'][i]
    video_name_list = video_file.split('/')[-1].split('.')
    video_name_list = video_name_list[:-1]
    video_name = ""
    for n in video_name_list:
        video_name += n
    # capturing the video from the given path
    capture = cv2.VideoCapture(video_file) 
    #frame rate
    frame_rate = capture.get(5)
    count = 0
    files = glob(test_frames_path + '/*')
    #removing all files from folder
    for f in files:
        os.remove(f)
    while(capture.isOpened()):
        #current frame number
        frame_id = capture.get(1) 
        read_correctly, frame = capture.read()
        if not read_correctly:
            break
        if (frame_id % math.floor(frame_rate) == 0):
            # storing the frames in a new folder named train_1
            filename = test_frames_path + "/" + video_name + "_frame{}_".format(count) + action +".jpg"
            count += 1
            cv2.imwrite(filename, frame)
    capture.release()
    
    # reading all the frames from temp folder
    images = glob(test_frames_path + '/*.jpg')
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img / 255
        prediction_images.append(img)
        
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)
    # predicting tags for each array
    prediction = np.argmax(model.predict(prediction_images), axis=-1)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y_train.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    actual.append(action)

100%|██████████████████████████████████████████████████████████████████████████████████| 44/44 [01:20<00:00,  1.82s/it]


In [60]:
y_train.columns.values[0]

'kick'

In [61]:
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

ModuleNotFoundError: No module named 'sklearn'

In [43]:
# for i in range(0, len(predict)):
#     print(predict[i] + " " + actual[i])

punch slap
slap slap
kick punch
kick kick
slap slap
slap slap
slap slap
kick kick
punch punch
slap slap
slap slap
slap slap
punch punch
slap slap
slap slap
kick kick
kick kick
slap slap
punch punch
punch punch
slap slap
punch slap
punch punch
slap slap
slap slap
kick kick
slap slap
slap slap
punch punch
kick kick
slap slap
punch punch
kick kick
slap slap
punch punch
slap slap
punch punch
slap slap
slap slap
punch punch
slap slap
punch punch
punch punch
kick kick
punch punch
kick kick
kick kick
punch punch
punch punch
kick kick
punch punch
slap slap
slap slap
slap slap
slap slap
slap slap
punch punch
slap slap
slap punch
punch punch
punch punch
kick kick
kick punch
punch punch
punch slap
slap slap
slap slap
slap slap
punch punch
kick kick
punch punch
slap slap
punch punch
slap slap
kick kick
kick kick
punch kick
punch punch
punch punch
slap slap
kick kick
kick kick
kick kick
kick kick
kick kick
kick kick
slap slap
slap slap
kick kick
kick kick
slap slap
slap slap
slap slap
punch punch
s